In [1]:
import pandas as pd
import numpy as np

In [26]:
churn=pd.read_csv("churn.csv")
customers=pd.read_csv("customers.csv")
payments=pd.read_csv("payments.csv")
usage=pd.read_csv("usage_data.csv")

In [27]:
print(customers.shape, usage.shape, payments.shape, churn.shape)
customers.info()
usage.describe()


(2000, 6) (15768, 5) (15768, 5) (359, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   customer_id        2000 non-null   object
 1   age                2000 non-null   int64 
 2   gender             2000 non-null   object
 3   city               2000 non-null   object
 4   signup_date        2000 non-null   object
 5   subscription_type  2000 non-null   object
dtypes: int64(1), object(5)
memory usage: 93.9+ KB


,minutes_watched,pages_viewed,activity_score
count,15768.000000,15768.000000,15768.000000
mean,53.898212,26.917364,65.983378
std,49.460454,28.507026,22.336994
min,0.000000,0.000000,0.000000
25%,20.000000,8.000000,50.200000
50%,34.000000,16.000000,66.300000
75%,73.000000,37.000000,83.300000
max,318.000000,200.000000,100.000000


In [40]:
usage['date'] = pd.to_datetime(usage['date'])
payments['payment_date']= pd.to_datetime(payments['payment_date'])

In [41]:
usage['month'] = usage['date'].dt.to_period('M')
payments['month'] = payments['payment_date'].dt.to_period('M')


In [68]:
usage_monthly = usage.groupby(['customer_id','month']).agg({
    'minutes_watched':'sum',
    'pages_viewed':'sum',
    'activity_score':'mean'
}).reset_index()


In [72]:
payments_monthly = payments.groupby(['customer_id','month']).agg({
    'amount':'sum',
    'status': lambda x: x.iloc[-1]  # last status in month
}).reset_index()


In [80]:
df = usage_monthly.merge(payments_monthly, on=['customer_id','month'], how='left')
df = df.merge(customers, on='customer_id', how='left')
df['month'] = df['month'].dt.to_timestamp()


In [96]:
churn_map = churn.set_index('customer_id')['churn_date']
df['churn_date'] = df['customer_id'].map(churn_map)


In [94]:
df['churn_flag'] = (
    (~df['churn_date'].isna()) &
    (pd.to_datetime(df['churn_date']) <= (df['month'] + pd.offsets.MonthEnd(0)))
)
df['churn_flag'] = df['churn_flag'].astype(int)


In [ ]:
# Average monthly usage
df['avg_monthly_usage'] = df.groupby('customer_id')['minutes_watched'].transform('mean')


In [86]:
# Usage drop percentage
df['usage_drop_pct'] = df.groupby('customer_id')['minutes_watched'].pct_change().fillna(0)


In [ ]:
#Drop vs Previous Month 
df['prev_month_minutes'] = df.groupby('customer_id')['minutes_watched'].shift(1)
df['drop_vs_prev'] = (df['prev_month_minutes'] - df['minutes_watched']) / (df['prev_month_minutes']+1)


In [ ]:
# Churn rate by subscription plan
churn_rate_by_plan = df.groupby('subscription_type').agg(
    total_customers=('customer_id','nunique'),
    churns=('churn_flag','sum')
).reset_index()

churn_rate_by_plan['churn_rate_pct'] = churn_rate_by_plan['churns'] / churn_rate_by_plan['total_customers'] * 100


In [88]:
# Revenue Lost
last_payments = payments.sort_values(['customer_id','payment_date']).groupby('customer_id').tail(1)
revenue_lost = last_payments[last_payments['customer_id'].isin(churn['customer_id'])]['amount'].sum()


In [91]:
churn.head(3)

,customer_id,churn_date,churn_reason
0,C100003,2025-06-19,No use
1,C100005,2025-10-26,Switch to competitor
2,C100011,2025-09-14,Switch to competitor


In [92]:
customers.head(3)

,customer_id,age,gender,city,signup_date,subscription_type
0,C100000,32,Female,Jaipur,2025-01-18,Standard
1,C100001,23,Male,Ghaziabad,2024-10-20,Premium
2,C100002,37,Female,New Delhi,2025-07-07,Basic


In [95]:
df.head(5)

,customer_id,month,minutes_watched,pages_viewed,activity_score,amount,status,age,gender,city,signup_date,subscription_type,churn_date,churn_flag,avg_monthly_usage,usage_drop_pct,prev_month_minutes,drop_vs_prev
0,C100000,2025-01-01,27,20,69.0,399,Success,32,Female,Jaipur,2025-01-18,Standard,NaN,0,46.3,0.000000,NaN,NaN
1,C100000,2025-02-01,36,58,86.5,399,Success,32,Female,Jaipur,2025-01-18,Standard,NaN,0,46.3,0.333333,27.0,-0.321429
2,C100000,2025-03-01,22,13,11.6,399,Success,32,Female,Jaipur,2025-01-18,Standard,NaN,0,46.3,-0.388889,36.0,0.378378
3,C100000,2025-04-01,41,43,55.4,399,Success,32,Female,Jaipur,2025-01-18,Standard,NaN,0,46.3,0.863636,22.0,-0.826087
4,C100000,2025-05-01,71,2,89.6,399,Success,32,Female,Jaipur,2025-01-18,Standard,NaN,0,46.3,0.731707,41.0,-0.714286


In [115]:
final_export = df.copy()
final_export.to_csv('analytic_table_for_powerbi.csv', index=False)